In [1]:
# Preprocess.py

import cv2
import numpy as np
import math


def preprocess(imgOriginal): #tham số truyền vào là ảnh gốc

    imgGrayscale = extractValue(imgOriginal) # LẤY ẢNH ĐEN TRẮNG
     
    imgMaxContrastGrayscale = maximizeContrast(imgGrayscale) 
    
    height, width = imgGrayscale.shape

    imgBlurred = np.zeros((height, width, 1), np.uint8) # tạo np chứa
    
    imgBlurred = cv2.GaussianBlur(imgMaxContrastGrayscale, (5,5), 0)
    


    imgThresh = cv2.adaptiveThreshold(imgBlurred, 255.0, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 19, 9)

    #Tạo ảnh nhị phân
    return imgGrayscale, imgThresh
    #Trả về ảnh xám và ảnh nhị phân



def extractValue(imgOriginal): # HÀM CHO RA ẢNH ĐEN TRẮNG
    height, width, numChannels = imgOriginal.shape
                  # numChannels: bộ màu sử dụng (red-green blue: 3, bw:1)
        
    imgHSV = np.zeros((height, width, 3), np.uint8)
    imgHSV = cv2.cvtColor(imgOriginal, cv2.COLOR_BGR2HSV)

    imgHue, imgSaturation, imgValue = cv2.split(imgHSV)
    
    #màu sắc, độ bão hòa, giá trị cường độ sáng
    #Không chọn màu RBG vì vd ảnh màu đỏ sẽ còn lẫn các màu khác nữa nên khó xđ ra "một màu" 
    
    return imgValue # ẢNH VALUE SẼ LÀ ẢNH ĐEN TRẮNG
    # end function


def maximizeContrast(imgGrayscale):
    #Làm cho độ tương phản lớn nhất 
    height, width = imgGrayscale.shape
    
    imgTopHat = np.zeros((height, width, 1), np.uint8)
    
    imgBlackHat = np.zeros((height, width, 1), np.uint8)
    
    structuringElement = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3)) #tạo bộ lọc kernel
    
    imgTopHat = cv2.morphologyEx(imgGrayscale, cv2.MORPH_TOPHAT, structuringElement, iterations = 10) #nổi bật chi tiết sáng trong nền tối
    #cv2.imwrite("tophat.jpg",imgTopHat)
    imgBlackHat = cv2.morphologyEx(imgGrayscale, cv2.MORPH_BLACKHAT, structuringElement, iterations = 10) #Nổi bật chi tiết tối trong nền sáng
    #cv2.imwrite("blackhat.jpg",imgBlackHat)
    imgGrayscalePlusTopHat = cv2.add(imgGrayscale, imgTopHat) 
    imgGrayscalePlusTopHatMinusBlackHat = cv2.subtract(imgGrayscalePlusTopHat, imgBlackHat)

    #Kết quả cuối là ảnh đã tăng độ tương phản 
    return imgGrayscalePlusTopHatMinusBlackHat
    # end function




